# Import Statements

---



In [0]:
# Data Manipulator Imports
import pandas as pd

# Data Retrieval Imports
from google.colab import files
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Normal Imports
import json
import os
import numpy as np
import pickle

# Import Data from Google Drive

---



In [0]:
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

## Don't need to run the following cell


---
Only run if data is already pickled!


In [0]:
# Run this if already pickled 
# Else don't run this block
# Bring Data in memory from Google Drive (CSV folder)

# Google Drive Link for Data (Professor given files, Pickled Files)
sharelink = "https://drive.google.com/open?id=1i-tyYUp4Ivzws-4tPtyMkdbTAVm2i2he"
fluff, gid = sharelink.split('=')

# Getting list of all files in the folder
file_list = drive.ListFile(
    {'q': "'{}' in parents".format(gid)}).GetList()

# Getting ID's of the Pickled files
xfileid = ''
yfileid = ''
for f in file_list:
  if(f['title']=='Xfileupdated'):
    xfileid = f['id']
  elif(f['title']=='Yfileupdated'):
    yfileid = f['id']

# Creating a local download path for Google Collab
local_download_path = os.path.expanduser('~/data')
try: os.makedirs(local_download_path)
except: pass

# Retrieving X_all data
fname = os.path.join(local_download_path, 'Xfileupdated')
downloaded = drive.CreateFile({'id':xfileid})
downloaded.GetContentFile(fname)
X_all = pickle.load(open(fname, 'rb'))

# Retrieving Y_all data
fname = os.path.join(local_download_path, 'Yfileupdated')
downloaded = drive.CreateFile({'id':yfileid})
downloaded.GetContentFile(fname)
Y_all = pickle.load(open(fname, 'rb'))

## Crunching the data

---



In [3]:
# Bring Data in memory from Google Drive (CSV folder)
# Google Drive Link for Data (Professor given files, Pickled Files)
sharelink = "https://drive.google.com/open?id=1i-tyYUp4Ivzws-4tPtyMkdbTAVm2i2he"
fluff, gid = sharelink.split('=')

# Getting list of all files in the folder
file_list = drive.ListFile(
    {'q': "'{}' in parents".format(gid)}).GetList()

# Creating local path for Google Collab
local_download_path = os.path.expanduser('~/data')
try:
  os.makedirs(local_download_path)
except: pass

# Getting all the files uploaded by the professor
dfs = []
signs = []
c = 0
for f in file_list:
  c += 1
  try:
    fname = os.path.join(local_download_path, f['title'])
    downloaded = drive.CreateFile({'id':f['id']})
    downloaded.GetContentFile(fname)

    # Converting into Dataframes
    df = pd.read_csv(fname)[["leftShoulder_x", "leftShoulder_y", "rightShoulder_x", "rightShoulder_y", "leftElbow_x", "leftElbow_y", "rightElbow_x", "rightElbow_y", "leftWrist_x", "leftWrist_y", "rightWrist_x", "rightWrist_y"]]

    # Creating master list of Dataframes
    dfs.append(df)

    # Sign from filename
    signs.append(f['title'].split('_')[0].lower())

    # Helper counter for progress
    print(c, end=' ')
  except:
    pass

3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 277 278 

In [8]:
print("minimum-{} & maximum-{} number of rows in a CSV input file.".format(min([len(i) for i in dfs]),max([len(i) for i in dfs])))

minimum-232 & maximum-232 number of rows in a CSV input file.


# Regularizing the Data

## Approach 1 [Not Followed]

---
Take the minimum length of a dataframe (limit set to 140) and cut down all other dataframes from the top and bottom to be the minimum length.


In [0]:
#Making All the Dataframes of the Minimum Length - Inplace
minlen = min([len(df) for df in dfs])
c = 0
for i in range(len(dfs)):
  c += 1
  dflen = len(dfs[i])
  if(dflen>minlen):
    dellen = dflen-minlen
    startlen = int(dellen/2)
    endlen = dflen - (dellen - startlen)
    dfs[i] = dfs[i].iloc[startlen:endlen].reset_index(drop=True)
  print(c, end=' ')

#This was used for testing while using our own videos as the training dataset

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 

## Approach 2 [ Followed ]


---
Getting the maximum length of rows in any given CSV input file, and imputing all other dataframes with mean value rows till they are the maximum length.


In [0]:
# Approach 2
maxlen = max([len(df) for df in dfs])
c = 0
for i in range(len(dfs)):
  c += 1
  dflen = len(dfs[i])
  #print(i)
  if(len(dfs[i])<maxlen):
    dfs[i] = dfs[i].append([pd.Series()]*(maxlen-dflen), ignore_index=True)
  dfs[i].fillna(dfs[i].mean(), inplace=True)

## Changing String Values of Labels to Integers

In [0]:
mapdict = {'communicate':0, 'communication':0, 'hope':1, 'mother':2, 'really':3, 'fun':4, 'buy':5}
for i in range(len(signs)):
  if(signs[i]=='communication'):
    signs[i] = ('communicate', mapdict[signs[i]])
  else:
    signs[i] = (signs[i], mapdict[signs[i]])

## Creating the master list of all above entries

In [0]:
X_all = np.stack(dfs)
y_all = np.array(signs)

## Storing the data

In [0]:
f = open('Yfileupdated', 'wb')
pickle.dump(y_all, f)
f.close()
f = open('Xfileupdated', 'wb')
pickle.dump(X_all, f)
f.close()

## Optional Code


---
Given a json object, should return a list of dicts with the entries.

If train=False, does not include the sign name (as there is none available in test dataset)



In [0]:
def json_to_list(jsonobj, train=False):
  jobj = json.loads(jsonobj)

  total_li = []
  tmpobj = {}

  if(train==True):
    filename = filename.replace('.json','')
    f = filename.split('_')
    tmpobj = {'sign':f[0], 'practice':f[2], 'person':f[3], 'imagenum':str(c)}
  
  c = 1
  baseval = None
  for image in jobj:
    for kp in image['keypoints']:
      tmpobj[kp['part']+'_score'] = kp['score']
      tmpobj[kp['part']+'_x'] = kp['position']['x']
      tmpobj[kp['part']+'_y'] = kp['position']['y']

    total_li.append(tmpobj)
    c += 1
  return total_li


## The End

---

